# Model M33


|Markets   |    A   |    B   |     W   |   TL  |  TK  |    |   CONS|
|----------|--------|--------|---------|-------|------|----|------ |
|PX        |  120   |        |  -120   |       |      |    |       |    
|PY        |        |  120   |  -120   |       |      |    |       |        
|PW        |        |        |   340   |       |      |    |  -340 | 
|PLS       |  -48   |  -72   |         |  120  |      |    |       |     
|PKS       |  -72   |  -48   |         |       |   120|    |       |       
|PL        |        |        |  -100   | -100  |      |    |200    |     
|PK        |        |        |         |       |  -100|    |100    |      
|TAX       |        |        |         |  -20  |   -20|    |  40   |    


In [1]:
using MPSGE
using DataFrames

In [ ]:
M33 = MPSGEModel()


txl0 = 20/100

@parameters(M33, begin
    TXL, txl0, (description = "Labor income tax rate",)
end)

@sectors(M33, begin
    X,	(description = " Activity level for sector X",)
    Y,	(description = " Activity level for sector Y",)
    W,	(description = " Activity level for sector W (Hicksian welfare index)",)
    TL,	(description = " Supply activity for L",)
    TK,	(description = " Supply activity for K",)
end)


@commodities(M33, begin
    PX,	(description = " Price index for commodity X",)
    PY,	(description = " Price index for commodity Y",)
    PL,	(description = " Price index for primary factor L (net of tax)",)
    PK,	(description = " Price index for primary factor K (net of tax)",)
    PLS,	(description = " Price index for primary factor L (gross of tax)",)
    PKS,	(description = " Price index for primary factor K (gross of tax)",)
    PW,	(description = " Price index for welfare (expenditure function)",)
end)

@consumer(M33, CONS, description = "Income level for consumer CONS")

@auxiliary(M33, TXK, description = "Endogenous captial tax from equal yield constraint")

@production(M33, X, [s=1,t=0], begin
    @output(PX, 120, t)
    @input(PLS, 40, s, reference_price = 1.2)
    @input(PKS, 60, s, reference_price = 1.2)
end)

@production(M33, Y, [s=1,t=0], begin
    @output(PY, 120, t)
    @input(PLS, 60, s, reference_price = 1.2)
    @input(PKS, 40, s, reference_price = 1.2)
end)

@production(M33, TL, [s=0,t=0], begin
    @output(PLS, 100, t , reference_price = 1.2)
    @input(PL, 100, s, taxes = [Tax(CONS, TXL)])
end)

@production(M33, TK, [s=0,t=0], begin
    @output(PKS, 100, t, reference_price = 1.2)
    @input(PK, 100, s, taxes = [Tax(CONS, TXK)])
end)

@production(M33, W, [s=.7, a=>s=1, t=0], begin
    @output(PW, 340, t)
    @input(PX, 120, a)
    @input(PY, 120, a)
    @input(PL, 100, s)
end)

@demand(M33, CONS, begin
    @final_demand(PW, 340)
    @endowment(PL, 200)
    @endowment(PK, 100)
end)

@aux_constraint(M33, TXK,
    TXL*PL*TL*100 + TXK*PK*TK*100 - 40 * (PX+PY)/2
);

# Benchmark

In [5]:
set_start_value(TXK, .2)
set_start_value(PLS, 1.2)
set_start_value(PKS, 1.2)

solve!(M33; cumulative_iteration_limit=0)
generate_report(M33)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_5AE.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 13

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.062000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Default price normalization using income for CONS - This value is fixed. Unfix with unfix(CONS).

Row,var,value,margin
,GenericV…,Float64,Float64
1,W,1.0,0.0
2,X,1.0,0.0
3,Y,1.0,0.0
4,PL,1.0,0.0
5,PW,1.0,0.0
6,PKS,1.2,0.0
7,PY,1.0,0.0
8,PX,1.0,0.0
9,PLS,1.2,0.0


# Counterfactual

In [8]:
set_silent(M33)

df = DataFrame([name => Float64[] for name in [:welfare,:labor_supply,:real_CONS, :captax]])

for s∈1:5
    set_value!(TXL, .25 - .05*s)
    
    solve!(M33)

    push!(df, [
        value(W), 
        value(TL),
        (value(PX)*value(X)*120 + value(PY)*value(Y)*120)/(value(PX)^.5*value(PY)^.5*240),
        value(TXK)
    ])
end
    

In [9]:
df

Row,welfare,labor_supply,real_CONS,captax
,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,0.2
2,1.00084,1.01609,1.00801,0.250389
3,1.00148,1.03287,1.0163,0.310753
4,1.00189,1.05038,1.02488,0.384325
5,1.00203,1.06869,1.03377,0.475907
